In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
# Read users csv file into Dataframes
users = pd.read_csv('takehome_users.csv',sep=',',encoding='latin-1')
users['last_session_creation_time']
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [4]:
users.isnull().sum()

object_id                        0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
dtype: int64

#### As of now lets keep last_session_creation_time - 3177 and invited_by_user_id  - 5583 as it is. will decide on later.

In [5]:
# Read users engagement csv file into Dataframes
users_log = pd.read_csv('takehome_user_engagement.csv',sep=',')
users_log.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [6]:
users_log.isnull().sum()

time_stamp    0
user_id       0
visited       0
dtype: int64

#### There are no null values

In [7]:
users_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [8]:
# Convert the time_stamp to datetime
users_log = pd.read_csv('takehome_user_engagement.csv', parse_dates=[0])
users_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [9]:
users_log.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [10]:
# Lets find out how many users are there
users_log['user_id'].nunique()

8823

In [11]:
# Lets find out if unique visited values
users_log['visited'].unique()

array([1], dtype=int64)

In [12]:
# Find out adopted users as per this defination - "adopted user" as a user who has 
# logged into the product on three separate days in at least one sevenday

# Set index to timestamp
users_log.set_index(users_log['time_stamp'], inplace=True)
users_log.head()

,time_stamp,user_id,visited
time_stamp,,,
2014-04-22 03:53:30,2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2013-12-25 03:45:04,2,1


In [13]:
# Drop the extra time_stamp
users_log.drop('time_stamp', axis=1, inplace = True)
users_log.head()

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1


In [14]:
users_log = users_log.groupby(by=['user_id', pd.Grouper(freq='W')]).sum()
#users_log = users_log.groupby(["user_id",pd.TimeGrouper('W')])["visited"].sum()

print(users_log)

                    visited
user_id time_stamp         
1       2014-04-27        1
2       2013-11-17        1
        2013-12-01        1
        2013-12-15        1
        2013-12-29        1
        2014-01-05        1
        2014-01-12        1
        2014-02-09        3
        2014-02-16        2
        2014-03-09        1
        2014-03-16        1
        2014-04-06        1
3       2013-03-24        1
4       2013-05-26        1
5       2013-01-27        1
6       2013-12-22        1
7       2012-12-23        1
10      2013-01-20        1
        2013-01-27        1
        2013-02-03        1
        2013-02-10        2
        2013-02-17        2
        2013-02-24        1
        2013-03-03        3
        2013-03-10        2
        2013-03-17        2
        2013-03-24        2
        2013-03-31        1
        2013-04-07        1
        2013-04-14        3
...                     ...
11982   2013-12-08        1
11983   2013-08-04        1
11984   2014-02-16  

In [15]:
users_log = users_log.groupby(level=0).visited.max().reset_index(name='max')

In [16]:
users_log['adoption'] = users_log['max'] >= 3
users_log['adoption'] = users_log['adoption'].astype(int)

users_log.head()

,user_id,max,adoption
0,1,1,0
1,2,3,1
2,3,1,0
3,4,1,0
4,5,1,0


In [17]:
# Merge the two tables users_log and users
users_log.set_index('user_id')
user_data = pd.merge(users_log, users, how='left', left_index=True, right_index=True)
user_data.index.name = 'user_id'
user_data.head()

,user_id,max,adoption,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
user_id,,,,,,,,,,,,,
0,1,1,0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,3,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,1,0,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,1,0,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,1,0,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [18]:
# Drop the columns which are not required
user_data.drop(['name','email','last_session_creation_time','invited_by_user_id','user_id'], axis = 1, inplace=True )
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8823 entries, 0 to 8822
Data columns (total 8 columns):
max                           8823 non-null int64
adoption                      8823 non-null int32
object_id                     8823 non-null int64
creation_time                 8823 non-null object
creation_source               8823 non-null object
opted_in_to_mailing_list      8823 non-null int64
enabled_for_marketing_drip    8823 non-null int64
org_id                        8823 non-null int64
dtypes: int32(1), int64(5), object(2)
memory usage: 517.1+ KB


In [19]:
user_data.head()

,max,adoption,object_id,creation_time,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id
user_id,,,,,,,,
0,1,0,1,2014-04-22 03:53:30,GUEST_INVITE,1,0,11
1,3,1,2,2013-11-15 03:45:04,ORG_INVITE,0,0,1
2,1,0,3,2013-03-19 23:14:52,ORG_INVITE,0,0,94
3,1,0,4,2013-05-21 08:09:28,GUEST_INVITE,0,0,1
4,1,0,5,2013-01-17 10:14:20,GUEST_INVITE,0,0,193


In [22]:
# creation_source column needs one-hot encode

user_data_enc = pd.get_dummies(user_data, columns = ['creation_source'])

# Lets drop creation_time as it is datetime datatype
user_data_enc.drop('creation_time', axis = 1, inplace = True)
user_data_enc.head()

,max,adoption,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
user_id,,,,,,,,,,,
0,1,0,1,1,0,11,1,0,0,0,0
1,3,1,2,0,0,1,0,1,0,0,0
2,1,0,3,0,0,94,0,1,0,0,0
3,1,0,4,0,0,1,1,0,0,0,0
4,1,0,5,0,0,193,1,0,0,0,0


In [34]:
# Recursive feature elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix


X = user_data_enc.drop('adoption', axis=1)
y =  user_data_enc['adoption']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

logreg = LogisticRegression()
selector = RFE(logreg, 5)
selector = selector.fit(X_train, y_train)
selector.support_
selector.ranking_


array([1, 6, 3, 4, 5, 1, 1, 2, 1, 1])

In [28]:
# Selected and ranked features
selected = pd.DataFrame({'Feature_Selected':selector.support_, 'Feature_Rank': selector.ranking_ }, index=X.columns)
selected = selected.sort_values('Feature_Rank')
selected


,Feature_Selected,Feature_Rank
max,True,1
creation_source_GUEST_INVITE,True,1
creation_source_ORG_INVITE,True,1
creation_source_SIGNUP,True,1
creation_source_SIGNUP_GOOGLE_AUTH,True,1
creation_source_PERSONAL_PROJECTS,False,2
opted_in_to_mailing_list,False,3
enabled_for_marketing_drip,False,4
org_id,False,5
object_id,False,6


#### Lower ranking is better, so top five features from above table are good predictors.


In [35]:
# Lets see the model score
print(selector.score(X_test, y_test))

y_pred = selector.predict(X_test)
print("{:30}{:3f}".format('F1 score', f1_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test accuracy', accuracy_score(y_test, y_pred)))

1.0
F1 score                      1.000000
Test accuracy                 1.000000


In [36]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

Confusion Matrix
[[1817    0]
 [   0  389]]
